In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df = pd.concat([utils.read_df_pkl('../input/histori*0*'),  utils.read_df_pkl('../input/new_mer*0*')], axis=0)

100%|██████████| 3/3 [00:00<00:00,  3.47it/s]


In [3]:
df['timezone'] = df['purchase_date'].map(lambda x: str(x)[11:13])
df['timezone'] = df['timezone'].astype('int8')
df['timezone'] = df['timezone'].map(lambda x: 
                                    'morning' if 5<=x<=10
                                    else 'noon' if 11<=x<=14
                                    else 'afternoon' if 15<=x<=18
                                    else 'night' if 19<=x<=23
                                    else 'midnight'
#                                     if x==24 or x<5
                                   )
df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,timezone
0,Y,C_ID_4e6213e9bc,88,N,0.0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1,16,37,afternoon
1,Y,C_ID_4e6213e9bc,88,N,0.0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1,16,16,noon
2,Y,C_ID_4e6213e9bc,88,N,0.0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1,16,37,night
3,Y,C_ID_4e6213e9bc,88,N,0.0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1,16,34,morning
4,Y,C_ID_4e6213e9bc,88,N,0.0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1,16,37,midnight


### 129 timezone feature

In [4]:
df['cnt'] = 1

df_tz = df.groupby([key, 'timezone'])['cnt'].sum().reset_index()
df_tz.name = 'timezone_cnt'
df_cnt = df.groupby([key])['cnt'].sum().reset_index()
df_cnt = df_cnt.merge(df_tz, how='inner', on=key)
df_cnt.head(10)

,card_id,cnt_x,timezone,cnt_y
0,C_ID_00007093c1,151,afternoon,60
1,C_ID_00007093c1,151,midnight,1
2,C_ID_00007093c1,151,morning,27
3,C_ID_00007093c1,151,night,26
4,C_ID_00007093c1,151,noon,37
5,C_ID_0001238066,149,afternoon,37
6,C_ID_0001238066,149,midnight,11
7,C_ID_0001238066,149,morning,17
8,C_ID_0001238066,149,night,48
9,C_ID_0001238066,149,noon,36


In [5]:
df_cnt['timezone_ratio'] = df_cnt['cnt_y'] /  df_cnt['cnt_x']
df_cnt = df_cnt[[key, 'timezone', 'timezone_ratio']]
df_cnt.set_index([key, 'timezone'], inplace=True)
df_tz_ratio = df_cnt.unstack()
df_tz_ratio.head(10)

timezone_ratio                                        
timezone             afternoon  midnight   morning     night      noon
card_id                                                               
C_ID_00007093c1       0.397351  0.006623  0.178808  0.172185  0.245033
C_ID_0001238066       0.248322  0.073826  0.114094  0.322148  0.241611
C_ID_0001506ef0       0.220588  0.014706  0.205882  0.044118  0.514706
C_ID_0001793786       0.299595  0.052632  0.089069  0.287449  0.271255
C_ID_000183fdda       0.258065  0.045161  0.051613  0.406452  0.238710
C_ID_00024e244b       0.300000  0.100000  0.171429  0.114286  0.314286
C_ID_0002709b5a       0.184211  0.144737  0.105263  0.092105  0.473684
C_ID_00027503e2       0.238095  0.047619  0.047619  0.023810  0.642857
C_ID_000298032a       0.709677  0.032258       NaN  0.129032  0.129032
C_ID_0002ba3c2e       0.191781  0.068493  0.164384  0.219178  0.356164

In [6]:
df_tz_ratio.columns = [f'timezone_ratio_{col[1]}' for col in df_tz_ratio.columns]
df_tz_ratio.head()

,timezone_ratio_afternoon,timezone_ratio_midnight,timezone_ratio_morning,timezone_ratio_night,timezone_ratio_noon
card_id,,,,,
C_ID_00007093c1,0.397351,0.006623,0.178808,0.172185,0.245033
C_ID_0001238066,0.248322,0.073826,0.114094,0.322148,0.241611
C_ID_0001506ef0,0.220588,0.014706,0.205882,0.044118,0.514706
C_ID_0001793786,0.299595,0.052632,0.089069,0.287449,0.271255
C_ID_000183fdda,0.258065,0.045161,0.051613,0.406452,0.238710


In [10]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train = df_train.merge(df_tz_ratio ,how='left', on=key)
df_test = df_test.merge(df_tz_ratio ,how='left', on=key)
df_feat = pd.concat([df_train, df_test], axis=0)
feat_cols = [col for col in df_feat.columns if col.count('timezone')]
df_feat = df_feat[[key] + feat_cols]
df_feat.fillna(0, inplace=True)
df_feat.head()

100%|██████████| 3/3 [00:00<00:00, 179.20it/s]


,card_id,timezone_ratio_afternoon,timezone_ratio_midnight,timezone_ratio_morning,timezone_ratio_night,timezone_ratio_noon
0,C_ID_92a2005557,0.247350,0.063604,0.151943,0.127208,0.409894
1,C_ID_3d0044924f,0.264045,0.070225,0.092697,0.283708,0.289326
2,C_ID_d639edf6cd,0.318182,0.000000,0.045455,0.545455,0.090909
3,C_ID_186d6a6901,0.464286,0.071429,0.059524,0.154762,0.250000
4,C_ID_cdbd2c0db2,0.195266,0.071006,0.165680,0.195266,0.372781


In [12]:
feat_no = '129_tzr'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_feat.columns:
    if col in ignore_features: continue
    if not(col.count('timezone')):continue
    feature = df_feat[col].astype('float32').values
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}_all_{col}@', obj=feature)